In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset

## Important Step:
please change the filename below to the file you want to use for training (This should not include the .csv)

In [23]:
# Specify the training file to take. Change the hashes, filename = '###'
filename = '40K'


# Load the CSV file
csv_file = './../3. Cleaned Data/'+filename+'.csv'
df = pd.read_csv(csv_file)

In [3]:
df.head()

,Unnamed: 0,original,word_count_original,corrected_fs,word_count_corrected_fs,word_count_diff,word_count_ratio,o_pos_tags,o_num_verbs,o_num_nouns,...,c_num_modal,num_verbs_diff,num_nouns_diff,num_adjs_diff,num_adv_diff,num_pronoun_diff,num_preposition_diff,num_conjunction_diff,num_article_diff,num_modal_diff
0,539741,"Since I have begun to live in London, I have b...",16,"Since I have begun living in London, I have be...",15,-1,0.937500,"[('Since', 'IN'), ('I', 'PRP'), ('have', 'VBP'...",7,2,...,0,-1,1,0,0,0,0,0,0,0
1,805208,"If she heard my description of her, she would ...",12,"If she had heard my description of her, she wo...",13,1,1.083333,"[('If', 'IN'), ('she', 'PRP'), ('heard', 'VBD'...",2,2,...,1,1,0,0,0,0,0,0,0,0
2,553823,I never can do.,4,I could never do it.,5,1,1.250000,"[('I', 'PRP'), ('never', 'RB'), ('can', 'MD'),...",1,0,...,1,0,0,0,0,1,0,0,0,0
3,792625,In the class I learned how to write paragraph ...,10,In the class I learned how to write paragraphs.,9,-1,0.900000,"[('In', 'IN'), ('the', 'DT'), ('class', 'NN'),...",2,2,...,0,0,0,-1,0,0,0,0,0,0
4,686799,I think imitating is important thing to learn ...,9,I think imitating is important to learning Eng...,8,-1,0.888889,"[('I', 'PRP'), ('think', 'VBP'), ('imitating',...",4,2,...,0,0,-1,0,0,0,0,0,0,0


In [4]:
# Change the column names in the dataframe
df.rename(columns = {'corrected_fs':'corrected'}, inplace = True)

In [5]:
# Split the dataset into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert the train and validation DataFrames to Hugging Face's Dataset instances
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [6]:
# Chose the model
model_name = 'gpt2'

In [7]:
# Assign cuda to the device to use for training
if torch.cuda.is_available(): 
 dev = "cuda:0" 
 print("This model will run on CUDA")
elif  torch.backends.mps.is_available(): 
 dev = "mps:0"
 print("This model will run on MPS")
else:
 dev = "cpu" 
 print("This model will run on CPU")
device = torch.device(dev) 

print(device)

This model will run on CUDA
cuda:0


In [8]:
# Load the tokenizer and the model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
config = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=config).to(device)

In [9]:
# Ensure that the tokenizer uses the same special tokens as GPT-2
tokenizer.pad_token = tokenizer.eos_token

# Function to tokenize and format input-output pairs
def tokenize_function(examples):
    inputs = [f"input: {orig} output: {corr}" for orig, corr in zip(examples["original"], examples["corrected"])]
    return tokenizer(inputs, padding=True, truncation=True, max_length=512, return_tensors='pt')

In [10]:
# Tokenize the train and validation data
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['original', 'corrected'])
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=['original', 'corrected'])

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [11]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=500,
    save_steps=500,
    warmup_steps=200,
    logging_dir="logs",
    evaluation_strategy="steps",
    logging_steps=100,
)

In [12]:
# Define a custom loss function to focus on the "output" tokens
def custom_loss_function(outputs, labels):
    shift_logits = outputs.logits[..., :-1, :].contiguous()
    shift_labels = labels[..., 1:].contiguous()
    loss = torch.nn.CrossEntropyLoss()(shift_logits, shift_labels)
    return loss

In [13]:
# Define a custom Trainer class that inherits from the original Trainer
class CustomTrainer(Trainer):
    
    # Override the compute_loss method to use a custom loss function
    def compute_loss(self, model, inputs, return_outputs=False):
        # Get the labels from the inputs dictionary and remove them from the inputs
        labels = inputs.pop("labels")
        
        # Get the model outputs by passing the inputs to the model
        outputs = model(**inputs)
        
        # Extract the logits from the model outputs
        logits = outputs.logits
        
        # Get the correct dimensions for the shift_labels tensor
        shift_labels = labels[..., 1:].reshape(-1)

        # Reshape the shift_logits tensor to align with the dimensions of the shift_labels tensor
        shift_logits = logits[..., :-1, :].reshape(-1, logits.size(-1))

        loss = torch.nn.CrossEntropyLoss()(shift_logits, shift_labels)

        if return_outputs:
            return loss, outputs
        
        # Otherwise, just return the loss
        return loss

# Create the custom Trainer with the custom loss function
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    
    # Use the DataCollatorForLanguageModeling to handle the data collation
    # Set mlm=False, as we are not using masked language modeling
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

In [14]:
print(model)
print(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)
cuda:0


In [15]:
# Train the model
trainer.train()

/home/timlinux/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/6750 [00:00<?, ?it/s]

{'loss': 3.1382, 'learning_rate': 2.5e-05, 'epoch': 0.04}
{'loss': 2.5698, 'learning_rate': 5e-05, 'epoch': 0.09}
{'loss': 2.5266, 'learning_rate': 4.923664122137405e-05, 'epoch': 0.13}
{'loss': 2.4917, 'learning_rate': 4.847328244274809e-05, 'epoch': 0.18}
{'loss': 2.4935, 'learning_rate': 4.7709923664122144e-05, 'epoch': 0.22}


  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 2.3650121688842773, 'eval_runtime': 2.9561, 'eval_samples_per_second': 676.575, 'eval_steps_per_second': 84.572, 'epoch': 0.22}
{'loss': 2.4111, 'learning_rate': 4.694656488549619e-05, 'epoch': 0.27}
{'loss': 2.4188, 'learning_rate': 4.618320610687023e-05, 'epoch': 0.31}
{'loss': 2.4324, 'learning_rate': 4.541984732824428e-05, 'epoch': 0.36}
{'loss': 2.4366, 'learning_rate': 4.465648854961832e-05, 'epoch': 0.4}
{'loss': 2.4007, 'learning_rate': 4.389312977099237e-05, 'epoch': 0.44}


...

  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 2.2761130332946777, 'eval_runtime': 2.9775, 'eval_samples_per_second': 671.713, 'eval_steps_per_second': 83.964, 'epoch': 2.67}
{'loss': 1.9875, 'learning_rate': 4.961832061068703e-06, 'epoch': 2.71}
{'loss': 1.9956, 'learning_rate': 4.198473282442748e-06, 'epoch': 2.76}
{'loss': 1.9862, 'learning_rate': 3.4351145038167944e-06, 'epoch': 2.8}
{'loss': 1.9942, 'learning_rate': 2.6717557251908397e-06, 'epoch': 2.84}
{'loss': 1.9888, 'learning_rate': 1.908396946564886e-06, 'epoch': 2.89}


  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 2.2746191024780273, 'eval_runtime': 2.928, 'eval_samples_per_second': 683.071, 'eval_steps_per_second': 85.384, 'epoch': 2.89}
{'loss': 1.9911, 'learning_rate': 1.1450381679389313e-06, 'epoch': 2.93}
{'loss': 2.0007, 'learning_rate': 3.816793893129771e-07, 'epoch': 2.98}
{'train_runtime': 464.6042, 'train_samples_per_second': 116.228, 'train_steps_per_second': 14.528, 'train_loss': 2.1862977984393086, 'epoch': 3.0}


TrainOutput(global_step=6750, training_loss=2.1862977984393086, metrics={'train_runtime': 464.6042, 'train_samples_per_second': 116.228, 'train_steps_per_second': 14.528, 'train_loss': 2.1862977984393086, 'epoch': 3.0})

In [24]:
# Save the trained model and tokenizer
output_dir = "../7. Models/"+filename+"/"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Then compress with this command: tar czvf trained_model.tar.gz trained_model/
# Upload to git/drive


('../7. Models/40K/tokenizer_config.json',
 '../7. Models/40K/special_tokens_map.json',
 '../7. Models/40K/vocab.json',
 '../7. Models/40K/merges.txt',
 '../7. Models/40K/added_tokens.json')

In [21]:
# Load trained model
output_dir = "../7. Models/"+filename+"/"
model = GPT2LMHeadModel.from_pretrained(output_dir)
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
